[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/03_modelado/01_modelos_kn_hmm.ipynb)
[![View on GitHub](https://img.shields.io/badge/View_on-GitHub-black?logo=github)](https://github.com/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/03_modelado/01_modelos_kn_hmm.ipynb)

# Modelos tradicionales: Kneser–Ney y HMM (T/S/D)
Este notebook implementa y evalúa dos modelos tradicionales para predicción de acordes:
- **Modelo n-grama con Kneser–Ney (trigrama)** sobre tokens funcionales (e.g., `I`, `ii`, `V7`, `bVII7`), con `<unk>` para raros y límites `<s>`, `</s>`.
- **HMM de funciones (T/S/D)** con estados ocultos **Tónica (T)**, **Subdominante (S)** y **Dominante (D)** derivados heurísticamente de la etiqueta funcional; emisiones son **tokens funcionales**.

**dataset**: `songdb_funcional_v4.csv`

**Contenido del notebook**:
1. Carga del dataset y preparación de datos.
2. Partición en train/val/test.
3. Implementación y ajuste del modelo n-grama con Kneser–Ney
4. Implementación y ajuste del HMM supervisado (T/S/D).
5. Evaluación comparativa de ambos modelos.


In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter, defaultdict
from pathlib import Path
from typing import List, Tuple, Dict

## 1) Cargamos el dataset
Leemos el CSV y preparamos las secuencias de tokens por canción. Usamos `title` + `composedby` como ID de canción.

In [2]:
# Configuración para acceder al dataset `songdb_funcional_v4.csv` en GitHub
USER = "antoniotrapote"
REPO = "chord-prediction-tfm"
BRANCH = "main"
PATH_IN_REPO = "anexos/data/songdb_funcional_v4.csv"
URL = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}/{PATH_IN_REPO}"

In [3]:
# Importamos el dataset de canciones con las progresiones funcionales
df = pd.read_csv(URL)

def tokenize_progression(prog: str):
    """
    Tokeniza una progresión en acordes separados
    """
    if pd.isna(prog):
        return []
    return [t for t in str(prog).strip().split() if t]

def build_sequences_by_song(df: pd.DataFrame):
    """
    Construye secuencias de acordes por canción a partir del DataFrame.
    """
    if "title" in df.columns and "composedby" in df.columns:
        song_ids = (df["title"].astype(str) + " — " + df["composedby"].astype(str)).tolist()
    elif "title" in df.columns:
        song_ids = df["title"].astype(str).tolist()
    else:
        song_ids = [f"song_{i}" for i in range(len(df))]
    seqs = {}
    for sid, prog in zip(song_ids, df["funcional_prog"].tolist()):
        seqs[sid] = tokenize_progression(prog)
    return seqs

seqs = build_sequences_by_song(df)
seqs = {k:v for k,v in seqs.items() if len(v) >= 3}
len(seqs), list(seqs)[:3]  # tamaño y primeras claves


(2591,
 ['Lullaby of Birdland — George Shearing',
  "It's A Most Unusual Day — Jimmy McHugh and HYarold Adamson",
  'Jump Monk — Charles Mingus'])

## 2) Partición train/val/test por canción
Usamos 80/10/10 con barajado determinista.

In [4]:

rng = np.random.default_rng(42)
song_list = list(seqs.keys())
rng.shuffle(song_list)

n = len(song_list)
n_train = int(n * 0.8)
n_val   = int(n * 0.1)
train_ids = song_list[:n_train]
val_ids   = song_list[n_train:n_train+n_val]
test_ids  = song_list[n_train+n_val:]

train_seqs = [seqs[sid] for sid in train_ids]
val_seqs   = [seqs[sid] for sid in val_ids]
test_seqs  = [seqs[sid] for sid in test_ids]

len(train_seqs), len(val_seqs), len(test_seqs)


(2072, 259, 260)

## 3) N-grama Kneser–Ney (trigrama)
Implementamos Kneser–Ney interpolado con descuento absoluto `D=0.75`. Mapeamos a `<unk>` tokens con frecuencia ≤ 1 en train.

El suavizado **Kneser–Ney (KN)** se diseñó para mejorar los modelos n-grama cuando hay **mucha dispersión** (en nuestro caso, muchos acordes posibles) y no siempre tenemos suficientes ejemplos de cada secuencia larga (como trigramas).

Tres cuestiones clave:
- **Descuento (D).**  
  A cada conteo de n-grama observado le restamos una pequeña cantidad fija (por ejemplo 0.75).  
  Así evitamos que los n-gramas muy frecuentes acaparen toda la probabilidad y liberamos un poco de “masa” para repartirla en otros casos.

- **Redistribución hacia *continuations*.**  
  La probabilidad “liberada” no se reparte de forma uniforme, sino en función de cuántos **contextos distintos** ha tenido un acorde.  
  Ejemplo: un acorde que aparece después de muchos acordes diferentes recibe más probabilidad que otro que solo aparece en un contexto concreto, aunque ambos tengan el mismo número de ocurrencias totales.  
  Esto hace que KN capte la **diversidad de contextos** en los que aparece un acorde, no solo su frecuencia absoluta.

- **Back-off (interpolación con modelos de menor orden).**  
  Si no encontramos un trigrama (tres acordes seguidos), en lugar de dar probabilidad cero, “retrocedemos” al modelo de bigramas. Si tampoco hay datos suficientes, retrocedemos al modelo de unigramas.  
  KN combina de forma ponderada (con un peso λ) la probabilidad descontada del n-grama con las probabilidades de los modelos de menor orden.

**En resumen:** Kneser–Ney reparte la probabilidad de manera más inteligente:  
quita un poco de probabilidad a los casos frecuentes y la asigna a los acordes que han demostrado poder aparecer en **más contextos diferentes**. Esto mejora mucho la calidad de las predicciones cuando el dataset es limitado.


**Hiperparámetros.**
- Orden \(N=3\) (trigrama).
- Descuento (típico: 0.75).
- Umbral `unk_threshold` para aplanar acordes raros a `<unk>`.

**Entrenamiento.**
1. Normaliza y tokeniza la progresión y añade `<s>` al inicio y `</s>` al final.
2. Cuenta n-gramas (uni-, bi- y trigramas).
3. Se registran los *continuations*, es decir, cuántos contextos distintos puede seguir cada acorde.
4. A partir de esos conteos se calculan los **pesos de retroceso (λ)**, que indican cuánta probabilidad hay que “devolver” a modelos más generales (bigramas y unigramas) cuando falta evidencia.

**Ventajas y límites.**
- Muy robusto en datos escasos; explicable.
- No modela dependencias largas > N.
- Sensible al vocabulario

In [ ]:
class KNTrigramLM:
    def __init__(self, discount: float = 0.75, unk_threshold: int = 1):
        self.D = discount # Descuento aplicado a los n-gramas más frecuentes
        self.unk_threshold = unk_threshold # umbral para mapear tokens raros a <unk>
        self.vocab = set() # vocabulario final (incluye <unk>, <s>, </s>)
        self.uni = Counter(); self.bi = Counter(); self.tri = Counter() # contadores de n-gramas
        self.continuation_counts = Counter() # nº de contextos únicos por token 
        self.bigram_continuation_counts = Counter() # nº de contextos únicos por bigrama 
        self.context_totals = Counter() 
        self.trigram_context_totals = Counter()
        self.fitted = False

    @staticmethod
    def add_bounds(seq):
        """Añade los tokens de inicio y fin de secuencia"""
        return ["<s>", "<s>"] + seq + ["</s>"]

    def fit(self, sequences):
        """
        Ajusta el modelo a las secuencias de entrenamiento.
        - Construye el vocabulario (con <unk> para acordes raros).
        - Añade tokens de inicio/fin (<s>, </s>).
        - Cuenta unigramas, bigramas y trigramas.
        - Calcula las estadísticas de continuación necesarias para Kneser–Ney.
        """
        # 1) Construcción del vocabulario
        token_counts = Counter(t for seq in sequences for t in seq)
        vocab = set([t for t,c in token_counts.items() if c > self.unk_threshold])
        vocab.update({"<s>", "</s>", "<unk>"}) # añadimos tokens especiales
        self.vocab = vocab

        def map_unk(seq):
            """Mapea tokens raros a <unk> """
            return [t if t in vocab else "<unk>" for t in seq]

        uni, bi, tri = Counter(), Counter(), Counter()
        left_contexts_for_w = defaultdict(set)
        left_contexts_for_bigram = defaultdict(set)

        #2) Recorrer secuencias, añadir bounds, mapear <unk> y contar n-gramas
        for seq in sequences:
            seq2 = self.add_bounds(map_unk(seq))
            for i in range(len(seq2)):
                w = seq2[i]; uni[w] += 1
                if i >= 1:
                    w1 = seq2[i-1]; bi[(w1, w)] += 1; left_contexts_for_w[w].add(w1)
                if i >= 2:
                    w2, w1 = seq2[i-2], seq2[i-1]
                    tri[(w2, w1, w)] += 1; left_contexts_for_bigram[(w1, w)].add(w2)

        # 3) Guardar conteos en el estado del modelo
        self.uni, self.bi, self.tri = uni, bi, tri
        for (w1, w), c in bi.items():
            self.context_totals[w1] += c
        for (w2, w1, w), c in tri.items():
            self.trigram_context_totals[(w2, w1)] += c

        # 4) Calcular los conteos de continuación
        self.continuation_counts = Counter({w: len(ctxs) for w, ctxs in left_contexts_for_w.items()})
        self.bigram_continuation_counts = Counter({(w1, w): len(ctxs) for (w1, w), ctxs in left_contexts_for_bigram.items()})
        self.total_unique_bigrams = sum(1 for _ in self.bi.keys())
        self.fitted = True

    def prob_unigram(self, w):
        """Calcula la probabilidad de continuacion a nivel de unigram"""
        cc = self.continuation_counts.get(w, 0)
        if self.total_unique_bigrams == 0:
            return 1.0 / max(1, len(self.vocab))
        return cc / self.total_unique_bigrams

    def prob_bigram(self, w_prev, w):
        """Calcula la probabilidad de continuacion a nivel de bigram.
        Incluye backoff a unigram si la probabilidad de bigram es 0."""
        c_wprev = self.context_totals.get(w_prev, 0)
        c_wprev_w = self.bi.get((w_prev, w), 0)
        if c_wprev > 0:
            lambda_wprev = (self.D * len([u for u in self.vocab if self.bi.get((w_prev, u), 0) > 0])) / c_wprev
        else:
            lambda_wprev = 1.0
        p_cont = self.prob_unigram(w)
        base = max(c_wprev_w - self.D, 0) / c_wprev if c_wprev > 0 else 0.0
        return base + lambda_wprev * p_cont

    def prob_trigram(self, w_prev2, w_prev, w):
        """Calcula la probabilidad de continuacion a nivel de trigram.
        Incluye backoff a bigram si la probabilidad de trigram es 0."""
        c_ctx = self.trigram_context_totals.get((w_prev2, w_prev), 0)
        c_trigram = self.tri.get((w_prev2, w_prev, w), 0)
        if c_ctx > 0:
            num_continuations = len([u for u in self.vocab if self.tri.get((w_prev2, w_prev, u), 0) > 0])
            lambda_ctx = (self.D * num_continuations) / c_ctx
        else:
            lambda_ctx = 1.0
        base = max(c_trigram - self.D, 0) / c_ctx if c_ctx > 0 else 0.0
        return base + lambda_ctx * self.prob_bigram(w_prev, w)

    def predict_ranking(self, history):
        """Dado un contexto (historia), devuelve una lista ordenada de
        tuplas (token, probabilidad) para cada token en el vocabulario."""
        hist = ["<s>", "<s>"] + [t if t in self.vocab else "<unk>" for t in history]
        w_prev2, w_prev = hist[-2], hist[-1]
        candidates = [w for w in self.vocab if w not in {"<s>"}]
        scores = []
        for w in candidates:
            p = self.prob_trigram(w_prev2, w_prev, w)
            scores.append((w, p))
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores

kn = KNTrigramLM(discount=0.75, unk_threshold=1)
kn.fit(train_seqs)
print(f"Tamaño del vocabulario: {len(kn.vocab)}")


Tamaño del vocabulario: 85


## 4) HMM (Hidden Markov Model) utilizando funciones (T/S/D)
Derivamos el estado oculto de cada token (`T`, `S`, `D`) a partir del **grado** del romano (e.g., `I, III, VI → T`; `ii, IV → S`; `V, vii → D`). Tratamos `♭II, ♭VI, ♭VII` como **S** y `♭III` como **T**. Estimamos **transiciones** y **emisiones** con suavizado add-k.

Modelamos la progresión como una cadena de **estados ocultos** de función armónica:
**Tónica (T)**, **Subdominante (S)** y **Dominante (D)**. 

**Cómo mapeamos un acorde a T/S/D (según el código):**
- Extraemos la base romana del token (antes de barras “/”) y detectamos bemol inicial (`b` o `♭`).  
- Mapeo por grados (mayormente diatónico):
  - **T (tónica)**: I, iii, vi y ♭III  
  - **S (subdominante)**: ii, IV y ♭II, ♭VI, ♭VII  
  - **D (dominante)**: V, vii  
  - En caso no cubierto, cae en **T** por defecto.
- Notas: `#`/`♯` no alteran la función en esta versión (solo distingimos bemol inicial). Dominantes secundarios y sustitutos se pierden en T/S/D.

**Entrenamiento:**
1. Para cada progresión, convertimos cada acorde a su función **T/S/D** con `function_from_token`.
2. Contamos:
   - **Iniciales**: frecuencia de cada estado inicial → π.  
   - **Transiciones** entre estados consecutivos → matriz **A**.  
   - **Emisiones** estado→acorde → matriz **B**.
3. Aplicamos **suavizado add-k** (`k=0.5` en nuestro experimento) a π, A y B para evitar ceros.  
   En **B** reservamos masa para `"<unk>"` (acordes fuera del vocabulario de *train*).

**Inferencia:**
- Si no hay historial: usamos **π** como distribución de estado.  
- Si hay historial: tomamos **el último acorde**, lo mapeamos a **T/S/D** y usamos **A[s_last]** como distribución del siguiente estado.  
- La probabilidad del **siguiente acorde** mezcla emisiones:  
  \( P(x_{t+1}) = \sum_{z \in \{T,S,D\}} P(x_{t+1}\mid z)\,P(z\mid \text{historia}) \).  
  Se rankean los acordes del vocabulario (se excluye `"<unk>"` de la salida final).

**¿Por qué T/S/D?**
- **Simplicidad y explicabilidad**: permite ver la lógica funcional (T→S→D) de forma compacta.  
- **Robustez con datos limitados**: menos parámetros que un HMM con muchos estados.  
  
**Limitaciones conocidas**
- **Pérdida de granularidad**: muchas funciones distintas (p. ej., V/ii, sustitutos por tritono, variantes maj7/m7) se colapsan en T/S/D → recomendaciones menos específicas.  
- **Alteraciones complejas**: no afectan la función en esta versión.  
- Con más tiempo, **ampliar estados** (p. ej., notación funcional explícita) podría **mejorar la competitividad** del HMM

In [ ]:

ROMAN_MAP = {"I":1,"II":2,"III":3,"IV":4,"V":5,"VI":6,"VII":7}

import re
def extract_roman_base(token: str):
    """Extrae grado del acorde (solo valores I..VII sin alteraciones)"""
    t = token.split('/')[0]
    m = re.match(r'^[b♭#♯]*([ivIV]+)', t)
    is_flat = bool(re.match(r'^[b♭]', t))
    if not m: return ("", is_flat)
    return (m.group(1).upper(), is_flat)

def degree_from_roman(r: str) -> int:
    """Convierte un numeral romano (I..VII) en un entero (1..7).
    Devuelve 0 si no es válido."""
    return ROMAN_MAP.get(r, 0)

def function_from_token(token: str) -> str:
    "Heurística para mapear un acorde a su función T/S/D"
    base, is_flat = extract_roman_base(token)
    deg = degree_from_roman(base)
    if is_flat and deg in {2,6,7}:  # bII, bVI, bVII
        return "S"
    if is_flat and deg == 3:        # bIII
        return "T"
    if deg in {1,3,6}:
        return "T"
    if deg in {2,4}:
        return "S"
    if deg in {5,7}:
        return "D"
    return "T"

STATES = ["T","S","D"]

from collections import Counter, defaultdict
class SupervisedHMM_TSD:
    """HMM supervisado con estados T/S/D."""
    def __init__(self, add_k: float = 1.0):
        self.add_k = add_k
        self.states = STATES
        self.A = None; self.B = None; self.pi = None # parámetros del modelo
        self.vocab = set() # acordes observados en el entrenamiento

    def fit(self, train_sequences):
        """
        Ajusta el HMM por conteo supervisado:
        1) Mapea cada acorde de la secuencia a T/S/D (function_from_token).
        2) Cuenta iniciales (π), transiciones (A) y emisiones (B).
        3) Aplica suavizado add-k y normaliza.
        """
        # Vocabulario de acordes observados (sin <unk> aquí; se añade en B)
        tok_counts = Counter(t for seq in train_sequences for t in seq)
        self.vocab = set(tok_counts.keys())

        # Estructuras de conteo
        trans = {s: Counter() for s in self.states} # A: conteos de transiciones estado→estado
        emit  = {s: Counter() for s in self.states} # B: conteos de emisiones estado→token
        pi_counts = Counter()                       # π: conteos de estados iniciales

        # Recorrer secuencias y contar
        for seq in train_sequences:
            if not seq: continue
            states_seq = [function_from_token(t) for t in seq]  # mapeo acorde→función
            pi_counts[states_seq[0]] += 1                       # primer estado de la secuencia
            for t, s in zip(seq, states_seq):                   # conteo de emisiones
                emit[s][t] += 1
            for s1, s2 in zip(states_seq[:-1], states_seq[1:]): # conteo de transiciones
                trans[s1][s2] += 1

        # Iniciales π con add-k
        total_pi = sum(pi_counts.values()) + self.add_k * len(self.states)
        self.pi = {s: (pi_counts[s] + self.add_k) / total_pi for s in self.states}

        # Transiciones A con add-k
        self.A = {}
        for s in self.states:
            total = sum(trans[s].values()) + self.add_k * len(self.states)
            self.A[s] = {s2: (trans[s][s2] + self.add_k) / total for s2 in self.states}

        # Emisiones B con add-k
        self.B = {}
        V = len(self.vocab)
        for s in self.states:
            total = sum(emit[s].values()) + self.add_k * (V + 1)
            self.B[s] = defaultdict(float)
            for w in self.vocab:
                self.B[s][w] = (emit[s][w] + self.add_k) / total
            self.B[s]["<unk>"] = self.add_k / total

    def predict_ranking(self, history):
        """
        Ranking de prob. para el siguiente acorde:
        - Si no hay historial → usa π como distribución de estado.
        - Si hay historial → mapea el último acorde a T/S/D y usa A[s_last] como dist. del estado siguiente.
        - Mezcla de emisiones: P(x) = sum_s P(x|s) * P(s|historia).
        - Devuelve lista (acorde, prob) ordenada desc. (sin "<unk>" en salida).
        """
        if len(history) == 0:
            state_probs = self.pi
        else:
            s_last = function_from_token(history[-1])
            state_probs = self.A[s_last]

        scores = {}
        for w in list(self.vocab) + ["<unk>"]:
            p = 0.0
            for s_next in self.states:
                p += state_probs[s_next] * self.B[s_next].get(w, self.B[s_next]["<unk>"])
            scores[w] = p
        scores.pop("<unk>", None)
        items = list(scores.items())
        items.sort(key=lambda x: x[1], reverse=True)
        return items

hmm = SupervisedHMM_TSD(add_k=0.5)
hmm.fit(train_seqs)
STATES, list(hmm.A.items())[:1]


(['T', 'S', 'D'],
 [('T',
   {'T': 0.44894101307819917,
    'S': 0.3055823463600941,
    'D': 0.24547664056170673})])

## 5) Evaluación (Top-k, MRR)
Medimos la calidad de **predicción del siguiente token** en test. Reportamos **Top@1/3/5** y **MRR**.

In [7]:

def evaluate_next_token_ranking(model, test_sequences, topk_list=(1,3,5)):
    total_positions = 0
    topk_hits = {k: 0 for k in topk_list}
    mrr_sum = 0.0
    for seq in test_sequences:
        for i in range(len(seq)-1):
            history = seq[:i+1]
            gold = seq[i+1]
            ranking = model.predict_ranking(history)
            ranks = {w: r+1 for r, (w, _) in enumerate(ranking)}
            rank_gold = ranks.get(gold, None)
            total_positions += 1
            if rank_gold is not None:
                for k in topk_list:
                    if rank_gold <= k:
                        topk_hits[k] += 1
                mrr_sum += 1.0 / rank_gold
    results = {
        "positions": total_positions,
        "MRR": mrr_sum / total_positions if total_positions > 0 else 0.0,
    }
    for k in topk_list:
        results[f"Top@{k}"] = topk_hits[k] / total_positions if total_positions > 0 else 0.0
    return results

class KNWrapperForEval:
    def __init__(self, kn_model):
        self.kn = kn_model
    def predict_ranking(self, history):
        hist = [t if t in self.kn.vocab else "<unk>" for t in history]
        return self.kn.predict_ranking(hist)

kn_res = evaluate_next_token_ranking(KNWrapperForEval(kn), test_seqs, topk_list=(1,3,5))
hmm_res = evaluate_next_token_ranking(hmm, test_seqs, topk_list=(1,3,5))
pd.DataFrame([{"Model":"Kneser–Ney (trigrama)", **kn_res},
              {"Model":"HMM funciones (T/S/D)", **hmm_res}])


,Model,positions,MRR,Top@1,Top@3,Top@5
0,Kneser–Ney (trigrama),12779,0.56052,0.415212,0.651616,0.737538
1,HMM funciones (T/S/D),12779,0.34576,0.191564,0.418890,0.512481


## Conclusiones

El modelo **Kneser–Ney (trigrama)** supera claramente al **HMM (T–S–D)** en todas las métricas de evaluación.

**Interpretación de las métricas:**
- **positions**: 12 779 puntos de predicción evaluados (idénticos para ambos modelos).
- **Top@k**: proporción de veces que el acorde real aparece entre las *k* mejores sugerencias del modelo.
  - **Top@1** (acierto exacto): 41.52% (KN) vs 19.15% (HMM).  
  - **Top@3**: 65.16% (KN) vs 41.89% (HMM).  
  - **Top@5**: 73.75% (KN) vs 51.24% (HMM).
- **MRR (Mean Reciprocal Rank)**: mide la posición media en la que aparece el acorde correcto dentro del ranking. Cuanto más cerca del primer lugar, mayor es el valor.  
  0.5605 (KN) vs 0.3457 (HMM) confirma el mejor ranking global del trigrama KN.

**Interpretación.**
- El **KN trigrama** es capaz de capturar **dependencias locales específicas entre acordes** y asignar probabilidades más precisas a las continuaciones típicas.  
- El **HMM con solo 3 estados (T/S/D)** pierde granularidad: muchos acordes diferentes se colapsan en la misma categoría funcional, lo que reparte la probabilidad de manera más difusa y reduce la precisión de las predicciones.

## 6) Roadmap (siguientes mejoras)
- Añadir **KN 4-grama** y optimización de `D` vía validación.
- Para HMM: probar **EM (Baum–Welch)** (peores resultados, vía descartada).
